In [1]:
#!pip install huffman

In [10]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import misc, ndimage
from scipy.fft import dctn, idctn
import cv2
import huffman
from collections import Counter

In [8]:
Q_jpeg = [[16, 11, 10, 16, 24, 40, 51, 61],
          [12, 12, 14, 19, 26, 28, 60, 55],
          [14, 13, 16, 24, 40, 57, 69, 56],
          [14, 17, 22, 29, 51, 87, 80, 62],
          [18, 22, 37, 56, 68, 109, 103, 77],
          [24, 35, 55, 64, 81, 104, 113, 92],
          [49, 64, 78, 87, 103, 121, 120, 101],
          [72, 92, 95, 98, 112, 100, 103, 99]]


def get_zig_zag(m):
    zigzag_template = [
        (0, 0), (0, 1), (1, 0), (2, 0), (1, 1), (0, 2), (0, 3), (1, 2),
        (2, 1), (3, 0), (4, 0), (3, 1), (2, 2), (1, 3), (0, 4), (0, 5),
        (1, 4), (2, 3), (3, 2), (4, 1), (5, 0), (6, 0), (5, 1), (4, 2),
        (3, 3), (2, 4), (1, 5), (0, 6), (0, 7), (1, 6), (2, 5), (3, 4),
        (4, 3), (5, 2), (6, 1), (7, 0), (7, 1), (6, 2), (5, 3), (4, 4),
        (3, 5), (2, 6), (1, 7), (2, 7), (3, 6), (4, 5), (5, 4), (6, 3),
        (7, 2), (7, 3), (6, 4), (5, 5), (4, 6), (3, 7), (4, 7), (5, 6),
        (6, 5), (7, 4), (7, 5), (6, 6), (5, 7), (6, 7), (7, 6), (7, 7)]

    zigzag_iteration = [m[i, j] for i, j in zigzag_template]
    return zigzag_iteration

In [18]:
X = misc.ascent()

# sarcina 1

def jpeg_encoding(X):
    r, c = X.shape
    r8 = int(np.ceil(r/8) * 8)
    c8 = int(np.ceil(c/8) * 8)
    
    padding_r = r8 - r 
    padding_c = c8 - c
    
    X_ext = cv2.copyMakeBorder(X, 0, padding_c, 0, padding_r, cv2.BORDER_CONSTANT, value=0)
    
    # y_nnz = 0
    # y_jpeg_nnz = 0
    Y_jpeg = np.zeros_like(X_ext)

    # luam pe rand fiecare bloc de 8x8 
    # ii facem dctn + cuantizare
    # parcurgem blocul in zig zag si concatenam raspunsurile de la fiecare bloc
    # aplicam huffman pe sirul concatenat

    zigzag_concatenat = []

    for i in range(int(r8/8)):
        for j in range(int(c8/8)):
            # extragerea blocului:
            x = X_ext[8*i:8*i + 8, 8*j:8*j + 8]

            # aplicarea dctn:
            y = dctn(x)
            # y_nnz += np.count_nonzero(y)

            # cuantizare:
            y_jpeg = Q_jpeg*np.round(y/Q_jpeg)
            # y_jpeg_nnz += np.count_nonzero(y_jpeg)

            # parcurgere zigzag:
            y_zigzag = get_zig_zag(y_jpeg)
            
            # concatenare rezultat:
            zigzag_concatenat.extend(y_zigzag)
            # Y_jpeg[8*i:8*i + 8, 8*j:8*j + 8] = y_jpeg

    # TODO: Huffman coding
    
    freq = Counter(zigzag_concatenat)
    huffman_code = huffman.codebook(freq.items())
   
    encoded_data = ''.join(huffman_code[num] for num in zigzag_concatenat)

    
    # decoded_data = huffman.decode(encoded_data, huffman_code)
    # Verificare: datele originale și decodificate ar trebui să fie identice
    # assert list(decoded_data) == data

    # return Y_jpeg
    return encoded_data, huffman_code



def jpeg_decoding(Y):
    r, c = Y.shape

    # TODO: Huffman decoding
    
    decoded = huffman.decode(encoded, huffman_code)
    
    X_jpeg = np.zeros_like(Y)
    for i in range(int(r/8)):
        for j in range(int(c/8)):
            y = Y[8*i:8*i + 8, 8*j:8*j + 8]
            x_jpeg = idctn(y)
            
            X_jpeg[8*i:8*i + 8, 8*j:8*j + 8] = x_jpeg
    
    return X_jpeg    
   

encoded_data, huffman_code = jpeg_encoding(X)
print(type(encoded_data[0]))

# TODO : SCJIMBA IMPLEMENTAREA HUFFMAN CU UNA CARE STIE CE FACE :(((((((

# print(len(Y_jpeg))
# X_jpeg = jpeg_decoding(Y_jpeg)


# plt.subplot(121).imshow(X, cmap=plt.cm.gray)
# plt.subplot(122).imshow(X_jpeg, cmap=plt.cm.gray)
# plt.show()



C:\Users\maria\AppData\Local\Temp\ipykernel_16312\1067549904.py:1: DeprecationWarning: scipy.misc.ascent has been deprecated in SciPy v1.10.0; and will be completely removed in SciPy v1.12.0. Dataset methods have moved into the scipy.datasets module. Use scipy.datasets.ascent instead.
  X = misc.ascent()


<class 'str'>
